In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.preprocessing import MinMaxScaler
import json
import itertools
from tqdm import tqdm
from __future__ import annotations
from scipy.sparse.linalg import expm
from sklearn.linear_model import Ridge
from typing import Sequence

In [ ]:
def lorenz_deriv(state, t, sigma=10.0, rho=28.0, beta=8.0/3.0):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x*(rho - z) - y
    dzdt = x*y - beta*z
    return [dxdt, dydt, dzdt]

def generate_lorenz_data(
    initial_state=[1.0, 1.0, 1.0],
    tmax=25.0,
    dt=0.01,
    sigma=10.0,
    rho=28.0,
    beta=8.0/3.0
):
    num_steps = int(tmax / dt) + 1 # +1 to include t=0
    t_vals = np.linspace(0, tmax, num_steps)
    sol = odeint(lorenz_deriv, initial_state, t_vals, args=(sigma, rho, beta))
    return t_vals, sol

In [ ]:
def compute_valid_prediction_time(y_true, y_pred, t_vals, threshold, lambda_max, dt):
    """
    Compute the Valid Prediction Time (VPT) and compare it to Lyapunov time T_lambda = 1 / lambda_max.
    
    Parameters
    ----------
    y_true : ndarray of shape (N, dim)
        True trajectory over time.
    y_pred : ndarray of shape (N, dim)
        Model's predicted trajectory over time (closed-loop).
    t_vals : ndarray of shape (N,)
        Time values corresponding to the trajectory steps.
    threshold : float, optional
        The error threshold, default is 0.4 as in your snippet.
    lambda_max : float, optional
        Largest Lyapunov exponent. Default=0.9 for Lorenz.
        
    Returns
    -------
    T_VPT : float
        Valid prediction time. The earliest time at which normalized error surpasses threshold
        (or the last time if never surpassed).
    T_lambda : float
        Lyapunov time = 1 / lambda_max
    ratio : float
        How many Lyapunov times the model prediction remains valid, i.e. T_VPT / T_lambda.
    """
    # 1) Average of y_true
    y_mean = np.mean(y_true, axis=0)  # shape (dim,)
    
    # 2) Time-averaged norm^2 of (y_true - y_mean)
    y_centered = y_true - y_mean
    denom = np.mean(np.sum(y_centered**2, axis=1))  # scalar
    
    # 3) Compute the normalized error delta_gamma(t) = ||y_true - y_pred||^2 / denom
    diff = y_true - y_pred
    err_sq = np.sum(diff**2, axis=1)  # shape (N,)
    delta_gamma = err_sq / denom      # shape (N,)
    
    # 4) Find the first time index where delta_gamma(t) exceeds threshold
    idx_exceed = np.where(delta_gamma > threshold)[0]
    if len(idx_exceed) == 0:
        # never exceeds threshold => set T_VPT to the final time
        T_VPT = t_vals[-1]
    else:
        T_VPT = t_vals[idx_exceed[0]]
    
    # 5) Compute T_lambda and ratio
    T_lambda = 1.0 / lambda_max

    # print(f"\n--- Valid Prediction Time (VPT) with threshold={threshold}, lambda_max={lambda_max} ---")

    T_VPT = (T_VPT - t_vals[0])  # Adjust T_VPT to be relative to the start time
    ratio = T_VPT / T_lambda

    return T_VPT, T_lambda, ratio

In [ ]:
import numpy as np
from sklearn.linear_model import Ridge
from numpy.linalg import eigvals

# ---------------------------------------------------------------------
# Utility helpers
# ---------------------------------------------------------------------
def scale_spectral_radius(W, desired_radius=0.95):
    """Affine-scale square matrix W so that its spectral radius equals desired_radius."""
    eigs = eigvals(W)
    current_radius = np.max(np.abs(eigs))
    if current_radius == 0:
        raise ValueError("Spectral radius of W is zero.")
    return W * (desired_radius / current_radius)

def augment_state_with_squares(x):
    """[x, x^2, 1]   (same convention as in CycleReservoir3D)."""
    return np.concatenate([x, x**2, [1.0]])

# ---------------------------------------------------------------------
# HH-LR class
# ---------------------------------------------------------------------
class HHLobeReservoir:
    """
    Hemispherically-Hierarchical Lobe Reservoir (HH-LR).

    Topology:
      * 8 anatomical modules  —  L/R × {F, P, T, O}
      * Intra-module: Watts–Strogatz small-world graphs (ring+rewire)
      * Intra-hemisphere inter-lobe: distance-modulated shortcuts
      * Inter-hemisphere (callosal): sparse homotopic bridges
    """

    # --- anatomical bookkeeping -------------------------------------------------
    _LOBES   = ['F', 'P', 'T', 'O']
    _HEMIS   = ['L', 'R']
    # rough 2-D centroids (arbitrary units) for distance computation
    _CENTROIDS = {
        ('L', 'F'): (-1.0,  1.0),
        ('L', 'P'): (-1.0,  0.0),
        ('L', 'T'): (-1.0, -1.0),
        ('L', 'O'): (-1.0, -2.0),
        ('R', 'F'): ( 1.0,  1.0),
        ('R', 'P'): ( 1.0,  0.0),
        ('R', 'T'): ( 1.0, -1.0),
        ('R', 'O'): ( 1.0, -2.0),
    }

    # ------------------------------------------------------------------
    def __init__(self,
                 reservoir_size=800,
                 input_dim=128,
                 spectral_radius=0.9,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 # small-world / shortcut hyper-parameters
                 k_ring=8,
                 p_rewire_frontal=0.30,
                 p_rewire_other=0.10,
                 P_lat=0.04,
                 sigma=5.0,
                 P_call=0.01,
                 seed=42):
        """
        Parameters
        ----------
        reservoir_size   : total neuron count (split equally across the 8 modules if not divisible)
        input_dim        : dimensionality of input vector u_t  (128 for EEG band-power features)
        leaking_rate     : α in leaky-integrator update
        """
        self.N     = reservoir_size
        self.D_in  = input_dim
        self.rho   = spectral_radius
        self.in_scale   = input_scale
        self.alpha = leaking_rate
        self.ridge_alpha = ridge_alpha
        self.seed  = seed

        # ------------------------------------------------------------------
        # 1) allocate neurons to the 8 modules as evenly as possible
        # ------------------------------------------------------------------
        base = self.N // 8
        counts = [base] * 8
        for i in range(self.N - base*8):
            counts[i] += 1
        self._module_slices = {}
        idx0 = 0
        for h in self._HEMIS:
            for l in self._LOBES:
                n = counts[len(self._module_slices)]
                self._module_slices[(h, l)] = slice(idx0, idx0 + n)
                idx0 += n

        # ------------------------------------------------------------------
        # 2) build adjacency matrix W according to HH-LR rules
        # ------------------------------------------------------------------
        rng = np.random.default_rng(self.seed)
        W = np.zeros((self.N, self.N), dtype=float)

        # 2.1 intra-module small-world wiring
        for (h, l), sl in self._module_slices.items():
            n_mod = sl.stop - sl.start
            k = min(k_ring, n_mod-1)  # guard tiny modules
            p_rewire = p_rewire_frontal if l == 'F' else p_rewire_other
            # ring lattice
            for i_local in range(n_mod):
                for m in range(1, k+1):
                    j_local = (i_local + m) % n_mod
                    i_glob = sl.start + i_local
                    j_glob = sl.start + j_local
                    W[i_glob, j_glob] = rng.standard_normal()
                    W[j_glob, i_glob] = rng.standard_normal()
            # rewire each existing edge with prob p_rewire
            for i_local in range(n_mod):
                for m in range(1, k+1):
                    if rng.random() < p_rewire:
                        # pick a random new target in same module (avoid self-loop)
                        j_local_new = rng.integers(0, n_mod-1)
                        if j_local_new >= i_local:
                            j_local_new += 1
                        i_glob = sl.start + i_local
                        j_glob_new = sl.start + j_local_new
                        # overwrite previous weight (both directions)
                        w_new = rng.standard_normal()
                        W[i_glob, j_glob_new] = w_new
                        W[j_glob_new, i_glob] = rng.standard_normal()

        # 2.2 intra-hemisphere inter-lobe shortcuts (distance-weighted)
        for h in self._HEMIS:
            for l1 in self._LOBES:
                for l2 in self._LOBES:
                    if l1 == l2:
                        continue
                    sl1 = self._module_slices[(h, l1)]
                    sl2 = self._module_slices[(h, l2)]
                    c1 = np.array(self._CENTROIDS[(h, l1)])
                    c2 = np.array(self._CENTROIDS[(h, l2)])
                    dist = np.linalg.norm(c1 - c2)
                    p_edge = P_lat * np.exp(-dist / sigma)
                    for i in range(sl1.start, sl1.stop):
                        mask = rng.random(sl2.stop - sl2.start) < p_edge
                        js = np.nonzero(mask)[0] + sl2.start
                        if js.size:
                            W[i, js] = rng.standard_normal(size=js.size)
                            W[js, i] = rng.standard_normal(size=js.size)

        # 2.3 inter-hemisphere callosal bridges (homotopic)
        for l in self._LOBES:
            sl_L = self._module_slices[('L', l)]
            sl_R = self._module_slices[('R', l)]
            for i in range(sl_L.start, sl_L.stop):
                mask = rng.random(sl_R.stop - sl_R.start) < P_call
                js = np.nonzero(mask)[0] + sl_R.start
                if js.size:
                    W[i, js] = rng.standard_normal(size=js.size)
                    W[js, i] = rng.standard_normal(size=js.size)

        # 2.4 spectral scaling
        W = scale_spectral_radius(W, self.rho)
        self.W = W

        # ------------------------------------------------------------------
        # 3) random input weights
        # ------------------------------------------------------------------
        rng = np.random.default_rng(self.seed + 1)
        self.W_in = (rng.random((self.N, self.D_in)) - 0.5) * 2.0 * self.in_scale

        # readout and state
        self.W_out = None
        self.x = np.zeros(self.N)

    # ------------------------------------------------------------------
    # ESN core methods (same signatures as CycleReservoir3D)
    # ------------------------------------------------------------------
    def reset_state(self):
        self.x = np.zeros(self.N)

    def _update(self, u):
        pre_activation = self.W @ self.x + self.W_in @ u
        x_new = np.tanh(pre_activation)
        self.x = (1.0 - self.alpha) * self.x + self.alpha * x_new

    def collect_states(self, inputs, discard=100):
        """
        Parameters
        ----------
        inputs  : iterable / array of shape (T, input_dim)
        discard : number of initial time-steps to omit from training
        """
        self.reset_state()
        states = []
        for u in inputs:
            self._update(u)
            states.append(self.x.copy())
        return np.array(states[discard:]), np.array(states[:discard])

    def fit_readout(self, train_input, train_target, discard=100):
        """
        Ridge regression read-out; identical augmentation as baseline.
        """
        states_use, _ = self.collect_states(train_input, discard=discard)
        targets_use = train_target[discard:]
        X_aug = np.vstack([augment_state_with_squares(s) for s in states_use])

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, targets_use)
        self.W_out = reg.coef_

    def predict_sequence(self, inputs):
        """
        Feed-forward prediction (no teacher forcing).  Suitable for classification:
        returns raw linear outputs; apply threshold/sigmoid externally.
        """
        preds = []
        self.reset_state()
        for u in inputs:
            self._update(u)
            preds.append(self.W_out @ augment_state_with_squares(self.x)) # quadratic lift-off
            #preds.append(self.W_out @ self.x)
        return np.array(preds)
    
    def predict_autoregressive(self, initial_input, n_steps):
        preds = []
        current_in = np.array(initial_input)
        for _ in range(n_steps):
            self._update(current_in)
            out = self.W_out @ augment_state_with_squares(self.x)
            preds.append(out)
            current_in = out
        return np.array(preds)

In [ ]:
grid = {
    "reservoir_size": [300],
    "input_dim": [3],
    "spectral_radius": [0.92],
    "input_scale": [0.8],
    "leaking_rate": [0.3, 0.5, 0.7, 0.9],
    "ridge_alpha": [1e-8],
    "k_ring": [6, 8, 10],
    "p_rewire_frontal": [0.2, 0.3, 0.4],
    "p_rewire_other": [0.05, 0.10, 0.2],
    "P_lat": [0.02, 0.04, 0.06],
    "sigma": [3, 5, 7],
    "P_call": [0.005, 0.01],
}

In [ ]:
def run_grid_search(model_class, param_grid, model_name,
                    output_path="grid_search_results.json"):
    combos = list(itertools.product(*param_grid.values()))
    param_keys = list(param_grid.keys())
    print(f"\n== Initial grid search for {model_name} with {len(combos)} combinations ==")

    results = []
    for comb in tqdm(combos, desc="Grid Search"):
        params = dict(zip(param_keys, comb))
        seed_scores = []
        
        for initial_state in [[1.0, 1.0, 1.0], [1.0, 2.0, 3.0], [2.0, 1.5, 4.0]]:
            tmax = 250
            dt   = 0.02
            t_vals, lorenz_traj = generate_lorenz_data(
                initial_state=initial_state,
                tmax=tmax,
                dt=dt
            )
            
            washout = 2000
            t_vals = t_vals[washout:]
            lorenz_traj = lorenz_traj[washout:]
            
            scaler = MinMaxScaler()
            scaler.fit(lorenz_traj)
            lorenz_traj = scaler.transform(lorenz_traj)
            
            T_data = len(lorenz_traj)
            for train_frac in [0.75, 0.8]:
                train_end = int(train_frac*(T_data-1))
                train_input  = lorenz_traj[:train_end]
                train_target = lorenz_traj[1:train_end+1]
                test_input   = lorenz_traj[train_end:-1]
                test_target  = lorenz_traj[train_end+1:]
                n_test_steps = len(test_input)
                initial_in = test_input[0]
                for seed in np.arange(1, 6):
                    model = model_class(**params, seed=seed)
                    model.fit_readout(train_input, train_target, discard=100)
                    preds = model.predict_autoregressive(initial_in, n_test_steps)
                    _, _, T_VPT_s = compute_valid_prediction_time(test_target, preds, t_vals, 0.4, 0.9, dt)
                    seed_scores.append(T_VPT_s)
        mean_score = float(np.mean(seed_scores))
        std_dev    = float(np.std(seed_scores))
        # is_stable  = std_dev < 1.5
        # status     = "Stable" if is_stable else "Unstable"
        
        # print(f"Params: {params} → Avg T_VPT={mean_score:.3f}, "
        #       f"Std Dev={std_dev:.3f} → {status}")

        results.append({
            "params":      params,
            "seed_scores": seed_scores,
            "mean_T_VPT":  mean_score,
            "std_dev":     std_dev,
            # "stable":      is_stable
        })

    # Save results
    with open(output_path, "w") as f:
        json.dump(results, f, indent=2)
    print(f"\nAll results saved to `{output_path}`")
    
    return results

In [ ]:
run_grid_search(HHLobeReservoir, grid, "HH-LR", output_path="hhlr[v1] 14.json")